In [ ]:
import time
from importlib import reload

import numpy as np

import hockey.hockey_env as h_env

In [ ]:
np.set_printoptions(suppress = True)

In [ ]:
reload(h_env)

# Normal Game Play

In [ ]:
env = h_env.HockeyEnv()

have a look at the initialization condition: alternating who starts and are random in puck position

In [ ]:
obs, info = env.reset()
obs_agent2 = env.obs_agent_two()
_ = env.render()

one episode with random agents

In [ ]:
obs, info = env.reset()
obs_agent2 = env.obs_agent_two()

for _ in range(600):
    env.render(mode = "human")
    a1 = np.random.uniform(-1, 1, 4)
    a2 = np.random.uniform(-1, 1, 4)
    obs, r, d, t, info = env.step(np.hstack([a1, a2]))
    obs_agent2 = env.obs_agent_two()
    if d or t: break

Without rendering, it runs much faster

"info" dict contains useful proxy rewards and winning information

In [ ]:
info, env.get_info_agent_two()

Winner == 0: draw

Winner == 1: you (left player)

Winner == -1: opponent wins (right player)

In [ ]:
env.close()

# Train Shooting

In [ ]:
env = h_env.HockeyEnv(mode = h_env.HockeyEnv.TRAIN_SHOOTING)

In [ ]:
o, info = env.reset()
_ = env.render()

for _ in range(50):
    env.render()
    a1 = [1, 0, 0, 1]  # np.random.uniform(-1,1,4)
    a2 = [0, 0., 0, 0]
    obs, r, d, t, info = env.step(np.hstack([a1, a2]))
    obs_agent2 = env.obs_agent_two()
    if d or t: break

In [ ]:
env.close()

# Train DEFENDING

In [ ]:
env = h_env.HockeyEnv(mode = h_env.HockeyEnv.TRAIN_DEFENSE)

In [ ]:
o, info = env.reset()
_ = env.render()

for _ in range(60):
    env.render()
    a1 = [0.1, 0, 0, 1]  # np.random.uniform(-1,1,3)
    a2 = [0, 0., 0, 0]
    obs, r, d, t, info = env.step(np.hstack([a1, a2]))
    print(r)
    obs_agent2 = env.obs_agent_two()
    if d or t: break

In [ ]:
env.close()

# Using discrete actions

In [ ]:
import random

In [ ]:
env = h_env.HockeyEnv(mode = h_env.HockeyEnv.TRAIN_SHOOTING)

In [ ]:
env.reset()
for _ in range(251):
    env.render()
    a1_discrete = random.randint(0, 7)
    a1 = env.discrete_to_continous_action(a1_discrete)
    a2 = [0, 0., 0, 0]
    obs, r, d, t, info = env.step(np.hstack([a1, a2]))
    obs_agent2 = env.obs_agent_two()
    if d or t: break

In [ ]:
env.close()

# Hand-crafted Opponent

In [ ]:
env = h_env.HockeyEnv()

In [ ]:
o, info = env.reset()
_ = env.render()
player1 = h_env.BasicOpponent(weak = False)
player2 = h_env.BasicOpponent()

In [ ]:
obs_buffer = []
reward_buffer = []
obs, info = env.reset()
obs_agent2 = env.obs_agent_two()
for _ in range(251):
    env.render()
    a1 = player1.act(obs)
    a2 = player2.act(obs_agent2)
    obs, r, d, t, info = env.step(np.hstack([a1, a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)
    obs_agent2 = env.obs_agent_two()
    if d or t: break
obs_buffer = np.asarray(obs_buffer)
reward_buffer = np.asarray(reward_buffer)

In [ ]:
np.mean(obs_buffer, axis = 0)

In [ ]:
np.std(obs_buffer, axis = 0)

If you want to use a fixed observation scaling, this might be a reasonable choice

In [ ]:
scaling = [1.0, 1.0, 0.5, 4.0, 4.0, 4.0,
           1.0, 1.0, 0.5, 4.0, 4.0, 4.0,
           2.0, 2.0, 10.0, 10.0, 4, 0, 4, 0]

In [ ]:
import matplotlib.pyplot as plt  # df: Changed from pylab, since its usage is deprecated

In [ ]:
plt.plot(obs_buffer[:, 2])
plt.plot(obs_buffer[:, 8])

In [ ]:
plt.plot(obs_buffer[:, 12])

In [ ]:
plt.plot(reward_buffer[:])

In [ ]:
np.sum(reward_buffer)

In [ ]:
info2 = env.get_info_agent_two()
info, info2, env.get_reward(info), env.get_reward_agent_two(info2)

In [ ]:
env.close()

# Human Opponent

In [ ]:
env = h_env.HockeyEnv()

In [ ]:
player1 = h_env.HumanOpponent(env = env, player = 1)
player2 = h_env.BasicOpponent()


In [ ]:
player1 = h_env.BasicOpponent()
player2 = h_env.HumanOpponent(env = env, player = 2)


In [ ]:
obs, info = env.reset()

env.render()
time.sleep(1)
obs_agent2 = env.obs_agent_two()
for _ in range(100):
    time.sleep(0.2)
    env.render()
    a1 = player1.act(obs)
    a2 = player2.act(obs_agent2)
    obs, r, d, _, info = env.step(np.hstack([a1, a2]))
    obs_agent2 = env.obs_agent_two()
    if d: break

In [ ]:
env.close()

# Check side consistency

In [ ]:
env = h_env.HockeyEnv()

In [ ]:
o, info = env.reset()
_ = env.render()
player1 = h_env.BasicOpponent(weak = False)
player2 = h_env.BasicOpponent(weak = False)

In [ ]:
obs_buffer = []
reward_buffer = []
obs2_buffer = []
winner_buffer = []
reward2_buffer = []
for game in range(1000):
    obs, info = env.reset()
    obs_agent2 = env.obs_agent_two()
    for _ in range(500):
        # env.render()
        a1 = player1.act(obs)
        a2 = player2.act(obs_agent2)
        #        a1 = np.random.uniform(-1,1,4)
        #        a2 = np.random.uniform(-1,1,4)
        obs, r, d, t, info = env.step(np.hstack([a1, a2]))
        info2 = env.get_info_agent_two()
        r2 = env.get_reward_agent_two(info2)
        obs_buffer.append(obs)
        obs_agent2 = env.obs_agent_two()
        obs2_buffer.append(obs_agent2)
        reward_buffer.append(r)
        reward2_buffer.append(r2)
        if d or t:
            winner_buffer.append(info["winner"])
            break
obs_buffer = np.asarray(obs_buffer)
reward_buffer = np.asarray(reward_buffer)
obs2_buffer = np.asarray(obs2_buffer)
reward2_buffer = np.asarray(reward2_buffer)

In [ ]:
obs_buffer.shape

In [ ]:
np.mean(obs_buffer, axis = 0)

In [ ]:
(np.std(obs_buffer, axis = 0) - np.std(obs2_buffer, axis = 0)) / np.std(obs_buffer, axis = 0)

In [ ]:
winner_buffer

In [ ]:
np.mean(winner_buffer, axis = 0)

In [ ]:
np.std(winner_buffer, axis = 0)

In [ ]:
np.sum(reward_buffer), np.sum(reward2_buffer)

In [ ]:
env.close()